In [3]:
import os
import tellurium as te
import numpy as np
import pandas as pd
import pickle

os.chdir('..')

In [29]:
with open('results/240707_VNMetGly_BIOSYN_train-test/train_myprob_test.obj', 'rb') as file:
    myprob = pickle.load(file)

r = te.loadSBMLModel(myprob.model[0])
# make upper and lower bounds
upper = np.array([r[i]*100 for i in [p for p in myprob.parameter_labels if 'Kcat' in p]] + 
                 [r[i]*100 for i in [p for p in myprob.parameter_labels if 'Km' in p]] + 
                 [r[i]*100 for i in [p for p in myprob.parameter_labels if 'Ki' in p]] +
                 [1 for _ in [p for p in myprob.parameter_labels if 'Gnc' in p]] +
                 [1 for _ in [p for p in myprob.parameter_labels if 'Gc' in p]] +
                 [100 for _ in [p for p in myprob.parameter_labels if 'v' in p]] + 
                 [100, 100])

lower = np.array([r[i]/100 for i in [p for p in myprob.parameter_labels if 'Kcat' in p]] + 
                 [r[i]/100 for i in [p for p in myprob.parameter_labels if 'Km' in p]] + 
                 [r[i]/100 for i in [p for p in myprob.parameter_labels if 'Ki' in p]] +
                 [1/100 for _ in [p for p in myprob.parameter_labels if 'Gnc' in p]] +
                 [1/100 for _ in [p for p in myprob.parameter_labels if 'Gc' in p]] +
                 [1/100 for _ in [p for p in myprob.parameter_labels if 'v' in p]] + 
                 [1/100, 1/100])

from src.fitting.pygmo_problems import SBMLGlobalFit_Multi_Fly
myprob = SBMLGlobalFit_Multi_Fly(
    model = myprob.model, 
    data = myprob.data, 
    parameter_labels=myprob.parameter_labels, 
    lower_bounds=lower, upper_bounds=upper, 
    metadata=myprob.metadata,
    variables=myprob.variables,
    scale=True,
    dlambda = 1)

In [30]:
with open('results/240707_VNMetGly_BIOSYN_train-test/checkpoint_t_test.dat', 'rb') as file:
    pops = pickle.load(file)

new_p_initial = np.array(myprob._unscale(pops[0].champion_x))
new_p_dict = {l:v for l,v in zip(myprob.parameter_labels,new_p_initial)}
new_upper = np.array([new_p_dict[i]*100 for i in [p for p in myprob.parameter_labels if 'Kcat' in p]] + 
                 [new_p_dict[i]*100 for i in [p for p in myprob.parameter_labels if 'Km' in p]] + 
                 [new_p_dict[i]*100 for i in [p for p in myprob.parameter_labels if 'Ki' in p]] +
                 [1 for _ in [p for p in myprob.parameter_labels if 'Gnc' in p]] +
                 [1 for _ in [p for p in myprob.parameter_labels if 'Gc' in p]] +
                 [new_p_dict[i]*100 for i in [p for p in myprob.parameter_labels if 'v' in p]] + 
                 [new_p_dict['rel0']*10, new_p_dict['rel1']*10])

new_lower = np.array([new_p_dict[i]/100 for i in [p for p in myprob.parameter_labels if 'Kcat' in p]] + 
                 [new_p_dict[i]/100 for i in [p for p in myprob.parameter_labels if 'Km' in p]] + 
                 [new_p_dict[i]/100 for i in [p for p in myprob.parameter_labels if 'Ki' in p]] +
                 [1/100 for _ in [p for p in myprob.parameter_labels if 'Gnc' in p]] +
                 [1/100 for _ in [p for p in myprob.parameter_labels if 'Gc' in p]] +
                 [new_p_dict[i]/100 for i in [p for p in myprob.parameter_labels if 'v' in p]] + 
                 [new_p_dict['rel0']/10, new_p_dict['rel1']/10])

myprob.set_bounds(new_upper, new_lower)
with open('results/240707_VNMetGly_BIOSYN_train-test/checkpoint_t_test2.dat', 'rb') as file:
    pops = pickle.load(file)

new_p_initial = np.array(myprob._unscale(pops[0].champion_x))
new_p_dict = {l:v for l,v in zip(myprob.parameter_labels,new_p_initial)}
new_upper = np.array([new_p_dict[i]*100 for i in [p for p in myprob.parameter_labels if 'Kcat' in p]] + 
                 [new_p_dict[i]*100 for i in [p for p in myprob.parameter_labels if 'Km' in p]] + 
                 [new_p_dict[i]*100 for i in [p for p in myprob.parameter_labels if 'Ki' in p]] +
                 [1 for _ in [p for p in myprob.parameter_labels if 'Gnc' in p]] +
                 [1 for _ in [p for p in myprob.parameter_labels if 'Gc' in p]] +
                 [new_p_dict[i]*100 for i in [p for p in myprob.parameter_labels if 'v' in p]] + 
                 [new_p_dict['rel0']*10, new_p_dict['rel1']*10])

new_lower = np.array([new_p_dict[i]/100 for i in [p for p in myprob.parameter_labels if 'Kcat' in p]] + 
                 [new_p_dict[i]/100 for i in [p for p in myprob.parameter_labels if 'Km' in p]] + 
                 [new_p_dict[i]/100 for i in [p for p in myprob.parameter_labels if 'Ki' in p]] +
                 [1/100 for _ in [p for p in myprob.parameter_labels if 'Gnc' in p]] +
                 [1/100 for _ in [p for p in myprob.parameter_labels if 'Gc' in p]] +
                 [new_p_dict[i]/100 for i in [p for p in myprob.parameter_labels if 'v' in p]] + 
                 [new_p_dict['rel0']/10, new_p_dict['rel1']/10])

myprob.set_bounds(new_upper, new_lower)
with open('results/240707_VNMetGly_BIOSYN_train-test/checkpoint_t_testw.dat', 'rb') as file:
    pops = pickle.load(file)

In [33]:
with open('src/frenda_brenda/Files/Reduced Complexity/240621_VN_Metabolism_Gly_BIOSYN.txt', 'r') as f:
    antimony = f.read()

x = [*list(myprob._unscale(pops[0].champion_x)), myprob.variables[0]['empty']['dilution_factor']]
for p,v in zip(myprob.parameter_labels+['dilution_factor'], x):
    start = antimony.find(p+' =')
    end = start + antimony[start:start+50].find(';')
    antimony = antimony.replace(antimony[start:end], p+' ='+str(v))

with open('results/240707_VNMetGly_BIOSYN_train-test/240621_VN_Metabolism_Gly_BIOSYN_fitted.sbml', 'w') as f:
    f.write(te.loada(antimony).getSBML())